In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys, os

#sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [3]:
import explore

# Import The Data

In [5]:
# Data that contains information about the completed orders

data_url = explore.get_data_url("data/nb.csv", "v0")
df_orders = pd.read_csv(data_url)

In [6]:
print("Orders data has {} rows and {} columns".format(df_orders.shape[0], df_orders.shape[1]))

Orders data has 536020 rows and 5 columns


In [7]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


In [8]:
# able that contains delivery requests by clients (completed and unfulfilled) and driver locations during request

data_url = explore.get_data_url("data/driver_locations_during_request.csv", "v0")
df_driversloc = pd.read_csv(data_url)

In [9]:
print("driver locations during request data has {} rows and {} columns".format(df_driversloc.shape[0], df_driversloc.shape[1]))

driver locations during request data has 1557740 rows and 8 columns


In [10]:
df_driversloc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


# Missing Values

In [11]:
explore.missing_values_table(df_orders)

Your selected dataframe has 5 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Trip Start Time,1651,0.3,object
Trip End Time,1,0.0,object


In [12]:
# Null start or end time might signify a non fulfilled order

df_orders['missing_dates'] = 0

indx = df_orders['Trip Start Time'].isna() | df_orders['Trip End Time'].isna()

df_orders.loc[indx, 'missing_dates'] = 1

In [17]:
# some values for 'Trip Origin' and 'Trip Destination' contain '1,1' which seem like a filler value

filler_inx = (df_orders['Trip Origin']=='1,1')|(df_orders['Trip Destination']=='1,1')

x = len(df_orders[filler_inx])

print("coordinates value that are filled by '1,1' are {}, which is {:.2f}% of total values".format(x, x/len(df_orders)))

coordinates value that are filled by '1,1' are 62, which is 0.00% of total values


In [18]:
# we can drop these values

df_orders.drop(df_orders[filler_inx].index, inplace= True)

In [19]:
explore.missing_values_table(df_driversloc)

Your selected dataframe has 8 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values,Dtype
created_at,1557740,100.0,float64
updated_at,1557740,100.0,float64


In [20]:
# two columns are empty. we drop them

df_driversloc.dropna(axis=1, inplace=True)
explore.missing_values_table(df_driversloc)

Your selected dataframe has 6 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values,Dtype


# Data Overview

We take a look at the data. Types of variables and calculate basic statistics

## 1. Data on Orders

In [21]:
df_orders.dtypes

Trip ID              int64
Trip Origin         object
Trip Destination    object
Trip Start Time     object
Trip End Time       object
missing_dates        int64
dtype: object

In [22]:
# Trip ID should all be unique values

df_orders['Trip ID'].nunique() == df_orders.shape[0]

True

In [25]:
# We divide the df on whether the order has missing dates

df_complete_orders = df_orders[df_orders['missing_dates']==0]

df_missing_orders = df_orders[df_orders['missing_dates']==1]

In [24]:
completed_order_id = df_complete_orders['Trip ID'].unique()
missingdates_order_id = df_missing_orders['Trip ID'].unique()

In [27]:
# We change the type of 
# 1. Time variables to datetime 
# 2. Origin and Destinations from strings to latitude and longitude


complete_orders = pd.DataFrame()

complete_orders['trip_id'] = df_complete_orders['Trip ID']

# change to datetime
complete_orders['trip_start_time'] = pd.to_datetime(df_complete_orders['Trip Start Time'])
complete_orders['trip_end_time'] = pd.to_datetime(df_complete_orders['Trip End Time'])

# split into latitude longitude
complete_orders['origin_lat'] = df_complete_orders['Trip Origin'].apply(lambda x: float(x.split(',')[0]))
complete_orders['origin_lng'] = df_complete_orders['Trip Origin'].apply(lambda x: float(x.split(',')[1]))

complete_orders['destination_lat'] = df_complete_orders['Trip Destination'].apply(lambda x: float(x.split(',')[0]))
complete_orders['destination_lng'] = df_complete_orders['Trip Destination'].apply(lambda x: float(x.split(',')[1]))

In [28]:
complete_orders.describe()

,trip_id,origin_lat,origin_lng,destination_lat,destination_lng
count,5.343060e+05,534306.000000,534306.000000,534306.000000,534306.000000
mean,9.901071e+05,6.528513,3.397394,6.523700,3.407750
std,3.454163e+05,0.091202,0.087272,0.093356,0.099802
min,3.919960e+05,4.859346,2.877515,4.812808,-0.293233
25%,6.929912e+05,6.450964,3.345037,6.447909,3.346085
50%,9.898230e+05,6.519571,3.374900,6.507310,3.382388
75%,1.279748e+06,6.594634,3.456603,6.593282,3.473291
max,1.637709e+06,9.218182,8.522293,9.056223,8.607887


In [29]:
print("Data on fulfilled orders spans the period from {} to {}".format(complete_orders['trip_start_time'].min(), 
complete_orders['trip_start_time'].max()))

Data on fulfilled orders spans the period from 2021-07-01 06:21:02 to 2022-06-13 10:04:28


In [30]:
# For orders that miss dates we change:
# - Origin and Destinations from strings to latitude and longitude


miss_orders = pd.DataFrame()

miss_orders['trip_id'] = df_missing_orders['Trip ID']

# split into latitude longitude
miss_orders['origin_lat'] = df_missing_orders['Trip Origin'].apply(lambda x: float(x.split(',')[0]))
miss_orders['origin_long'] = df_missing_orders['Trip Origin'].apply(lambda x: float(x.split(',')[1]))

miss_orders['destination_lat'] = df_missing_orders['Trip Destination'].apply(lambda x: float(x.split(',')[0]))
miss_orders['destination_long'] = df_missing_orders['Trip Destination'].apply(lambda x: float(x.split(',')[1]))

In [31]:
miss_orders.describe()

,trip_id,origin_lat,origin_long,destination_lat,destination_long
count,1.652000e+03,1652.000000,1652.000000,1652.000000,1652.000000
mean,1.293932e+06,7.221046,3.786585,7.219593,3.789560
std,2.788812e+05,0.380039,0.214001,0.380467,0.215525
min,3.931930e+05,6.422819,3.192377,6.405968,3.125112
25%,1.249252e+06,7.364181,3.860555,7.350632,3.857426
50%,1.368064e+06,7.410068,3.890328,7.405548,3.890128
75%,1.471976e+06,7.428348,3.908526,7.433988,3.911522
max,1.637091e+06,7.533172,4.008998,7.543277,4.011485


## 2. Data on Drivers Locations

In [33]:
# The column 'id' doesn't seem to indicate anything aside from being a simple index

df_driversloc['id'].unique()

array([      1,       2,       3, ..., 1557738, 1557739, 1557740])

In [34]:
# dropping the column 'id' reveal that the data have duplicates

clean_driversloc = df_driversloc.drop(columns=['id']).drop_duplicates()

clean_driversloc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 718826 entries, 0 to 1557731
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       718826 non-null  int64  
 1   driver_id      718826 non-null  int64  
 2   driver_action  718826 non-null  object 
 3   lat            718826 non-null  float64
 4   lng            718826 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 32.9+ MB


In [35]:
print("{:.2f}% of drivers data was duplicated.".format(1-len(clean_driversloc)/len(df_driversloc)))

0.54% of drivers data was duplicated.


## 3. Unfulfilled Orders

We are looking for unfulfilled orders within the data. There are a couple of possibilities:

1. order_id values missing from the table for completed orders.
2. order_id values for which no "accepted" driver_action exists
3. 'trip id' values that had no associated 'start trip time'

### 1. order_id values missing from the table for completed orders.

In [36]:
driver_orderid = clean_driversloc['order_id'].unique()
len(driver_orderid)

26492

In [53]:
# order_id values missing from the table for completed orders

not_completed_1 = list(set(driver_orderid)-set(completed_order_id))
len(not_completed_1)

39

### 2. order_id values for which no "accepted" driver_action exists

In [44]:
# orders that hase be accepted at least once
accepted_orders = clean_driversloc.loc[clean_driversloc['driver_action']=='accepted', 'order_id'].unique()
len(accepted_orders)

25881

In [61]:
# orders that were never accepted (in our table)

not_completed_2 = list(set(driver_orderid)-set(accepted_orders))
len(not_completed_2)

611

### 3. 'trip id' values that had no associated 'start trip time'

In [69]:
not_completed_3 = missingdates_order_id

### Discussion

If all assumptions are correct and not in contradition, the not_completed_1, not_completed_2 and not_completed_3 will be subsets of the unfulfilled orders.

Here we look if there are contradtictions:

In [59]:
# Was any of the  orders missing from completed order table ever accepted by drivers

test = list(set(not_completed_1) - set(accepted_orders)) 
print("is our test successful? ", len(test)==len(not_completed_1))

is our test successful?  False


In [63]:
# Was any of the orders that were never accepted in drivers table mentioned as completed in complete_order table

test = list(set(not_completed_2) - set(completed_order_id)) 
print("is our test successful? ", len(test)==len(not_completed_2))

is our test successful?  False


In [64]:
test = list(set(not_completed_1) - set(missingdates_order_id)) 
len(test)

5

There is a contradiction between definition 1 and 2 as well as a significant overlap between definition 1 and 3.

We conclude -we choose- to take definition 1 and 3 as different representation of unfulfilled order


**unfulfilled orders are the order which are not included in the complete_order table.**

## 4. Label Unfulfilled Orders

In [76]:
clean_driversloc['fulfilled'] =clean_driversloc['order_id'].apply(lambda x: 0 if x in not_completed_1 else 1)

# Write Data

In [83]:
complete_orders.to_csv('../data/complete_orders.csv', index=False)

In [84]:
miss_orders.to_csv('../data/missing_orders.csv', index=False)

In [85]:
clean_driversloc.to_csv('../data/clean_driver_locations.csv', index=False)